In [1]:
# Import libraries and packages for the project 
# import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup#pulling data out of HTML files
from time import sleep#slow down the execution to pass the security check
import csv
import pandas as pd
import numpy as np
import re
import chromedriver_autoinstaller
chromedriver_autoinstaller.install()

'/Users/jchen056/opt/anaconda3/lib/python3.9/site-packages/chromedriver_autoinstaller/109/chromedriver'

# load data and filter out CUNY students

In [2]:
df_o4=pd.read_csv('output4.csv')
df_o3=pd.read_csv('output3.csv')
df_o2=pd.read_csv('output2.csv')
df_o1=pd.read_csv('output1.csv')

# df_o5=pd.read_csv('output5.csv')
# df_o6=pd.read_csv('output6.csv')
# df_o7=pd.read_csv('output7.csv')
# df_o8=pd.read_csv('output8.csv')
# df_o9=pd.read_csv('output9.csv')
# df_o10=pd.read_csv('output10.csv')
# df_o11=pd.read_csv('output11.csv')
#df_o=pd.concat([df_o3,df_o2,df_o1,df_o4,df_o5,df_o6,df_o7,df_o8,df_o9,df_o10,df_o11])
df_o=pd.concat([df_o3,df_o2,df_o1,df_o4])
# df_o=pd.concat([df_o5,df_o6,df_o7,df_o8,df_o9,df_o10,df_o11])
df_o.reset_index(drop=True,inplace=True)
df_o

,Name,Job Title,Location,Degrees,Schools,Work Places,Work Titles,URL
0,Minerva Francis,Postdoctoral Fellow in Addiction Research and ...,"New York, New York, United States","['\nDoctor of Education - EdD, Health and Beha...","['\nTeachers College, Columbia UniversityTeach...",['\nCUNY Hostos Community CollegeCUNY Hostos C...,['\nAdjunct Assistant ProfessorAdjunct Assista...,https://www.linkedin.com/in/minerva-francis-9a...
1,Minerva Francis,--,United States,[],[],[],[],https://www.linkedin.com/in/minerva-francis-b7...
2,Minerva Francis,--,New York City Metropolitan Area,[],[],[],[],https://www.linkedin.com/in/minerva-francis-66...
3,Minerva Frances,--,"New York, New York, United States",[],[],[],[],https://www.linkedin.com/in/minerva-frances-b7...
4,Minerva Boufrancis chahine,--,"Beirut, Beirut Governorate, Lebanon","['\nBT3, AccountingBT3, Accounting\n']",['\nInstitut st AntoineInstitut st Antoine\n'],[],[],https://www.linkedin.com/in/minerva-boufrancis...
...,...,...,...,...,...,...,...,...
303,Marvin (Gene) Bennett,♦ Experienced professional,"Salina, Kansas, United States","[""\nAssociate's degree, PsychologyAssociate's ...",['\nCloud County Community CollegeCloud County...,"['\nSalina USD 305Salina USD 305\n', '\nPlanet...","['\nBuilding Operator Building Operator \n', '...",https://www.linkedin.com/in/marvin-gene-bennet...
304,Marvin Bennett,Chief Executive Officer at AA DIAMONDS Multina...,Los Angeles Metropolitan Area,[],[],['\nAA DIAMONDS Multinational AA DIAMONDS Mult...,['\nChief Executive OfficerChief Executive Off...,https://www.linkedin.com/in/marvin-bennett-599...
305,"Marvin C. Bennett Sr., P.E., RS","Principal, Metro Engineering Services","Rockville, Maryland, United States","['\nMasters, Civil EngineeringMasters, Civil E...","['\nLehigh UniversityLehigh University\n', '\n...",['\nMorgan State University · Part-timeMorgan ...,['\nProfessor of PracticeProfessor of Practice...,https://www.linkedin.com/in/mbennettsr?miniPro...
306,Giselle Acco,Activity Coordinator at City University of New...,"Brooklyn, New York, United States","['\nMaster, chemical engineerMaster, chemical ...",['\nThe City College of New YorkThe City Colle...,['\nCity University of New York - Borough of m...,['\nActivity CoordinatorActivity Coordinator\n...,https://www.linkedin.com/in/giselle-acco-ba0a5...


In [2]:
campuses=['CUNY New York City College of Technology','College of Staten Island',
          'Lehman College','LaGuardia Community College',
          
          'Baruch College', 'Queens College',
           'Queensborough Community College',
          
          'Borough of Manhattan Community College','CUNY Bronx Community College',
          'Kingsborough Community College',
          
          'CUNY York College',
          'CUNY Hostos Community College', 'Hunter College',
          'Medgar Evers College, CUNY',
          'John Jay College (CUNY)',
         'Brooklyn College',
          'The City College of New York',
         'The Graduate Center, City University of New York']

reg_campus=''
for i in campuses:
    reg_campus+=i
    reg_campus+='|'
reg_campus=reg_campus[0:-1]
reg_campus

'CUNY New York City College of Technology|College of Staten Island|Lehman College|LaGuardia Community College|Baruch College|Queens College|Queensborough Community College|Borough of Manhattan Community College|CUNY Bronx Community College|Kingsborough Community College|CUNY York College|CUNY Hostos Community College|Hunter College|Medgar Evers College, CUNY|John Jay College (CUNY)|Brooklyn College|The City College of New York|The Graduate Center, City University of New York'

In [53]:
df_o['CUNY students']=df_o['Schools'].str.contains(reg_campus,regex=True)
CUNY_students=df_o[df_o['CUNY students']==True]
CUNY_students['Name'].value_counts()
CUNY_students.shape

/var/folders/c3/wqq2zhw90l596jqv25j63ggc0000gn/T/ipykernel_65899/3884603010.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_o['CUNY students']=df_o['Schools'].str.contains(reg_campus,regex=True)


(52, 9)

# data cleaning

In [24]:
def clean_edu(A):
    B=A[1:-1]
    C=B.replace("'\\n","")
    
    #print(C)
    E=C.split("\\n', ")
    #print(E)
    F=[]
    for num in range(len(E)):
        i=E[num]
        if num==len(E)-1:
            j=i[0:int(len(i)/2)-1]
            F.append(j)
        else:
            j=i[int(len(i)/2):]
            F.append(j)
    return F

CUNY_students['edu']=CUNY_students['Schools'].apply(clean_edu)

/var/folders/c3/wqq2zhw90l596jqv25j63ggc0000gn/T/ipykernel_65899/2872166999.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CUNY_students['edu']=CUNY_students['Schools'].apply(clean_edu)


In [55]:
def clean_degree(A):
    B=A[1:-1]#remove the square brackets
    C=B.replace("'\\n","")#remove the starting ''\\n for each degree
    E=C.split("\\n', ")#split based on "\\n,"
    
    F=[]
    for num in range(len(E)):
        i=E[num]
        if num==len(E)-1:
            j=i[0:int(len(i)/2)-1]
            F.append(j)
        else:
            j=i[int(len(i)/2):]
            F.append(j)
    return F




/var/folders/c3/wqq2zhw90l596jqv25j63ggc0000gn/T/ipykernel_65899/98565977.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CUNY_students['deg']=CUNY_students['Degrees'].apply(clean_degree)


In [56]:
CUNY_students['titles']=CUNY_students['Work Titles'].apply(clean_degree)

/var/folders/c3/wqq2zhw90l596jqv25j63ggc0000gn/T/ipykernel_65899/1531349801.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CUNY_students['titles']=CUNY_students['Work Titles'].apply(clean_degree)


In [57]:
CUNY_students['work']=CUNY_students['Work Places'].apply(clean_degree)

/var/folders/c3/wqq2zhw90l596jqv25j63ggc0000gn/T/ipykernel_65899/4132574622.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CUNY_students['work']=CUNY_students['Work Places'].apply(clean_degree)


In [58]:
CUNY_students['school']=CUNY_students['Schools'].apply(clean_degree)

/var/folders/c3/wqq2zhw90l596jqv25j63ggc0000gn/T/ipykernel_65899/1219393710.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CUNY_students['school']=CUNY_students['Schools'].apply(clean_degree)


In [59]:
CUNY=CUNY_students[['Name','deg','school','work','titles','URL']]
CUNY.reset_index()

,index,Name,deg,school,work,titles,URL
0,0,Abdelhamid jnane,"[Doctor of Philosophy (Ph.D.), Clinical Labora...","[Rutgers University, The City College of New Y...","[City College, CCNY]",[Research Assistant],https://www.linkedin.com/in/abdelhamid-jnane-0...
1,7,"Alejandra Castano, PhD","[Doctor of Philosophy - PhD, Chemistry. Concen...","[Stony Brook University, Queens College]","[5 yrs 3 mos, Full-time, Full-time, 6 yrs 11 m...",[Grant Assistant for the STEP Queensborough Br...,https://www.linkedin.com/in/alejandra-castano?...
2,16,Amarachi ikwuagwu,"[Doctor of Philosophy - PhD, Educational Psych...","[Northcentral University, Keller Graduate Scho...","[Con Edison, Kraft Foods]","[Engineer, Engineering Inetrn]",https://www.linkedin.com/in/amarachi-ikwuagwu-...
3,36,Benedette Adewale,"[Doctor of Philosophy (Ph.D.), Chemistry, Mast...","[Stony Brook University, The City College of N...",[State University of New York College at Old W...,"[Lecturer, Graduate Research Assistant, Underg...",https://www.linkedin.com/in/benedette-adewale-...
4,37,Bertholin Pierre,"[Master of Arts in Teaching Science, Biology, ...","[Brooklyn College, Medgar Evers College, CUNY]","[Kingsborough Community College, Accelerated S...",[Supplemental Math Instructor (Teaching pre- a...,https://www.linkedin.com/in/bertholin-pierre-3...
5,39,Carlos Jaramillo,"[Doctor of Philosophy (Ph.D.), Computer Scienc...","[The Graduate Center, City University of New Y...","[Sea Machines Robotics · Full-time, Piaggio Fa...","[Senior Autonomy Engineer, Senior Robotics Eng...",https://www.linkedin.com/in/carlos-jaramillo-c...
6,46,"Carmen I. Carrión, PsyD","[Master of Arts (M.A.), Clinical Psychology, M...","[Roosevelt University, Brooklyn College]","[4 yrs 5 mos, University of Michigan, 6 yrs 1 ...","[Postdoctoral Fellow, Neuropsychology Intern, ...",https://www.linkedin.com/in/carmencarrion?mini...
7,51,Charlie Corredor,"[Doctor of Philosophy (PhD), Chemical Engineer...","[University of Washington, University of Washi...","[Microsoft, Facebook, OtoNexus Medical Technol...","[Senior Product Manager, Product Strategy Mana...",https://www.linkedin.com/in/charlescorredor?mi...
8,56,Charren C. Cabaroy,"[Master of Arts (MA), Geology/Earth Science, G...","[Brooklyn College, York College, Geology]","[USEPA · Full-time, Hydro Environmental Ltd, A...","[Physical Scientist, Assistant Project Manager...",https://www.linkedin.com/in/charren-c-cabaroy-...
9,57,Christian Paylor-Smith,"[Master’s Degree, Secondary Education and Teac...","[Brooklyn College, Brooklyn College, Brooklyn ...",[South Country Central School District · Full-...,"[High School Science Teacher, Middle School Sc...",https://www.linkedin.com/in/christianpaylorsmi...


In [19]:
def combine_two_columns(A,B):
    C=[]
    if len(A)==0 or len(B)==0 or len(A)!=len(B):
        return C
    else:
        for i in range(len(A)):
            a=A[i]+' @ '+B[i]
            print(a)
            C.append(a)
    
        return C


In [61]:
CUNY['edu hist']=CUNY.apply(lambda x: combine_two_columns(x.deg, x.school), axis=1)

Doctor of Philosophy - PhD, Chemistry. Concentration: Biological Chemistry @ Stony Brook University
B.A., Organic Chemistry @ Queens College
Doctor of Philosophy - PhD, Educational Psychology @ Northcentral University
MBA, Project Management @ Keller Graduate School of Management of DeVry University
Msc, Chemical Engineering @ The City College of New York
Doctor of Philosophy (Ph.D.), Chemistry @ Stony Brook University
Master’s Degree, Chemistry @ The City College of New York
Bachelor’s Degree, Chemistry @ City University of New York-College of Staten Island
Doctor of Philosophy (Ph.D.), Computer Science @ The Graduate Center, City University of New York
Master’s Degree, Computer Science @ The City College of New York
Bachelor’s Degree, Computer Engineering @ The City University of New York
Master of Arts (M.A.), Clinical Psychology @ Roosevelt University
Master of Arts (M.A.), Experimental Psychology @ Brooklyn College
Doctor of Philosophy (PhD), Chemical Engineering, Nanotechnology &

/var/folders/c3/wqq2zhw90l596jqv25j63ggc0000gn/T/ipykernel_65899/3561426280.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CUNY['edu hist']=CUNY.apply(lambda x: combine_two_columns(x.deg, x.school), axis=1)


In [62]:
CUNY['exp hist']=CUNY.apply(lambda x: combine_two_columns(x.titles, x.work), axis=1)

Research Assistant @ City College, CCNY
Engineer @ Con Edison
Engineering Inetrn @ Kraft Foods
Supplemental Math Instructor (Teaching pre- and algebra) @ Kingsborough Community College
Supplemental  Math Instructor @ Accelerated Study in Associate Programs (ASAP) at Kingsborough Community College
Senior Autonomy Engineer @ Sea Machines Robotics · Full-time
Senior Robotics Engineer  @ Piaggio Fast Forward · Full-time
Perception Engineer @ Aurora Flight Sciences Corporation
Research Assistant @ The City College of New York
Research Intern @ Mitsubishi Electric Research Laboratories
Senior Product Manager @ Microsoft
Product Strategy Manager @ Facebook
Vice President Operations @ OtoNexus Medical Technologies
Co-Founder, CEO @ Phoresa
National Science Foundation Research Fellow @ University of Washington
Physical Scientist @ USEPA · Full-time
Assistant Project Manager @ Hydro Environmental Ltd
Geologist/ Data Management Analyst @ Andalusia Petroleum Service, Inc.
Research Assistant @ Brid

/var/folders/c3/wqq2zhw90l596jqv25j63ggc0000gn/T/ipykernel_65899/515747134.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CUNY['exp hist']=CUNY.apply(lambda x: combine_two_columns(x.titles, x.work), axis=1)


In [63]:
CUNY

,Name,deg,school,work,titles,URL,edu hist,exp hist
0,Abdelhamid jnane,"[Doctor of Philosophy (Ph.D.), Clinical Labora...","[Rutgers University, The City College of New Y...","[City College, CCNY]",[Research Assistant],https://www.linkedin.com/in/abdelhamid-jnane-0...,[],"[Research Assistant @ City College, CCNY]"
7,"Alejandra Castano, PhD","[Doctor of Philosophy - PhD, Chemistry. Concen...","[Stony Brook University, Queens College]","[5 yrs 3 mos, Full-time, Full-time, 6 yrs 11 m...",[Grant Assistant for the STEP Queensborough Br...,https://www.linkedin.com/in/alejandra-castano?...,"[Doctor of Philosophy - PhD, Chemistry. Concen...",[]
16,Amarachi ikwuagwu,"[Doctor of Philosophy - PhD, Educational Psych...","[Northcentral University, Keller Graduate Scho...","[Con Edison, Kraft Foods]","[Engineer, Engineering Inetrn]",https://www.linkedin.com/in/amarachi-ikwuagwu-...,"[Doctor of Philosophy - PhD, Educational Psych...","[Engineer @ Con Edison, Engineering Inetrn @ K..."
36,Benedette Adewale,"[Doctor of Philosophy (Ph.D.), Chemistry, Mast...","[Stony Brook University, The City College of N...",[State University of New York College at Old W...,"[Lecturer, Graduate Research Assistant, Underg...",https://www.linkedin.com/in/benedette-adewale-...,"[Doctor of Philosophy (Ph.D.), Chemistry @ Sto...",[]
37,Bertholin Pierre,"[Master of Arts in Teaching Science, Biology, ...","[Brooklyn College, Medgar Evers College, CUNY]","[Kingsborough Community College, Accelerated S...",[Supplemental Math Instructor (Teaching pre- a...,https://www.linkedin.com/in/bertholin-pierre-3...,[],[Supplemental Math Instructor (Teaching pre- a...
39,Carlos Jaramillo,"[Doctor of Philosophy (Ph.D.), Computer Scienc...","[The Graduate Center, City University of New Y...","[Sea Machines Robotics · Full-time, Piaggio Fa...","[Senior Autonomy Engineer, Senior Robotics Eng...",https://www.linkedin.com/in/carlos-jaramillo-c...,"[Doctor of Philosophy (Ph.D.), Computer Scienc...",[Senior Autonomy Engineer @ Sea Machines Robot...
46,"Carmen I. Carrión, PsyD","[Master of Arts (M.A.), Clinical Psychology, M...","[Roosevelt University, Brooklyn College]","[4 yrs 5 mos, University of Michigan, 6 yrs 1 ...","[Postdoctoral Fellow, Neuropsychology Intern, ...",https://www.linkedin.com/in/carmencarrion?mini...,"[Master of Arts (M.A.), Clinical Psychology @ ...",[]
51,Charlie Corredor,"[Doctor of Philosophy (PhD), Chemical Engineer...","[University of Washington, University of Washi...","[Microsoft, Facebook, OtoNexus Medical Technol...","[Senior Product Manager, Product Strategy Mana...",https://www.linkedin.com/in/charlescorredor?mi...,"[Doctor of Philosophy (PhD), Chemical Engineer...","[Senior Product Manager @ Microsoft, Product S..."
56,Charren C. Cabaroy,"[Master of Arts (MA), Geology/Earth Science, G...","[Brooklyn College, York College, Geology]","[USEPA · Full-time, Hydro Environmental Ltd, A...","[Physical Scientist, Assistant Project Manager...",https://www.linkedin.com/in/charren-c-cabaroy-...,"[Master of Arts (MA), Geology/Earth Science, G...","[Physical Scientist @ USEPA · Full-time, Assis..."
57,Christian Paylor-Smith,"[Master’s Degree, Secondary Education and Teac...","[Brooklyn College, Brooklyn College, Brooklyn ...",[South Country Central School District · Full-...,"[High School Science Teacher, Middle School Sc...",https://www.linkedin.com/in/christianpaylorsmi...,"[Master’s Degree, Secondary Education and Teac...",[High School Science Teacher @ South Country C...


In [65]:
CUNY.to_csv('cleaned_scholar_CUNY.csv')

# another dataset

In [17]:
df_missingPPL=pd.read_csv('l1_missingPPL.csv')
CUNY_l1_m=df_missingPPL

In [15]:
# #they are all CUNY students according to Dr Brathwaite, no need to apply the CUNY filtering
# df_missingPPL['CUNY students']=df_missingPPL['Schools'].str.contains(reg_campus,regex=True)
# CUNY_l1_m=df_missingPPL[df_missingPPL['CUNY students']==True]
# CUNY_l1_m['Name'].value_counts()
# CUNY_l1_m

/var/folders/c3/wqq2zhw90l596jqv25j63ggc0000gn/T/ipykernel_10825/1300895579.py:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_missingPPL['CUNY students']=df_missingPPL['Schools'].str.contains(reg_campus,regex=True)


,Name,Job Title,Location,Degrees,Schools,Work Places,Work Titles,URL,CUNY students
1,Vaughn-Clai Marsh,Attorney with a strong risk management and eng...,"Scottsdale, Arizona, United States","['\nJuris Doctor, LawJuris Doctor, Law\n', '\n...",['\nNew England Law | BostonNew England Law | ...,['\nBryan Cave Leighton Paisner LLP · Full-tim...,['\nCompliance and Conflicts AttorneyComplianc...,https://www.linkedin.com/in/vaughn-clai-marsh-...,True
3,Ana Alas Iglesias,Research Aministrator at Baruch College | Pre-...,New York City Metropolitan Area,"['\nMaster of Arts (M.A.), EconomicsMaster of ...","['\nThe New SchoolThe New School\n', '\nBaruch...",['\nBaruch College · Full-timeBaruch College ·...,['\nSponsored Programs CoordinatorSponsored Pr...,https://www.linkedin.com/in/aalasiglesias,True
4,Angela Padilla,Student at The City College of New York,"Brooklyn, New York, United States","['\nMaster of Science - MS, Physics Teacher Ed...",['\nThe City College of New YorkThe City Colle...,[],[],https://www.linkedin.com/in/angela-padilla-985...,True
6,Anthony Cruz-Balberdy,Senior Scientist at Ventus Therapeutics,"New York, New York, United States","['\nDoctor of Philosophy (Ph.D.), Chemistry - ...","['\nThe Graduate Center, City University of Ne...",['\nVentus Therapeutics · Full-timeVentus Ther...,"['\nSenior ScientistSenior Scientist\n', '\nSu...",https://www.linkedin.com/in/anthony-cruz-balbe...,True
7,Andrew Wills,Consultant at Brooklyn Public Library,"Brooklyn, New York, United States","['\nBachelor’s Degree, Applied MathematicsBach...",['\nCUNY New York City College of TechnologyCU...,['\nBrooklyn Public LibraryBrooklyn Public Lib...,"['\nConsultantConsultant\n', '\nTutorTutor\n',...",https://www.linkedin.com/in/andrew-wills-67a1a289,True


In [18]:
def clean_degree(A):
    B=A[1:-1]#remove the square brackets
    C=B.replace("'\\n","")#remove the starting ''\\n for each degree
    E=C.split("\\n', ")#split based on "\\n,"
    
    F=[]
    for num in range(len(E)):
        i=E[num]
        if num==len(E)-1:
            j=i[0:int(len(i)/2)-1]
            F.append(j)
        else:
            j=i[int(len(i)/2):]
            F.append(j)
    return F

CUNY_l1_m['deg']=CUNY_l1_m['Degrees'].apply(clean_degree)
CUNY_l1_m['titles']=CUNY_l1_m['Work Titles'].apply(clean_degree)
CUNY_l1_m['work']=CUNY_l1_m['Work Places'].apply(clean_degree)
CUNY_l1_m['school']=CUNY_l1_m['Schools'].apply(clean_degree)

CUNY_m1=CUNY_l1_m[['Name','deg','school','work','titles','URL']]
CUNY_m1.reset_index()
CUNY_m1

,Name,deg,school,work,titles,URL
0,Lori Collins,[],[],"[US Army Futures Command, CCDC · Full-time]",[Computer Engineer],https://www.linkedin.com/in/lori-collins-a6411...
1,Vaughn-Clai Marsh,"[Juris Doctor, Law, Masters, Chemical Engineer...","[New England Law | Boston, The City College of...","[Bryan Cave Leighton Paisner LLP · Full-time, ...","[Compliance and Conflicts Attorney, Senior Con...",https://www.linkedin.com/in/vaughn-clai-marsh-...
2,Allie O.,[],[],[],[],https://www.linkedin.com/in/allieorellana
3,Ana Alas Iglesias,"[Master of Arts (M.A.), Economics, Bachelor of...","[The New School, Baruch College, Borough of Ma...",[Baruch College · Full-time],[Sponsored Programs Coordinator],https://www.linkedin.com/in/aalasiglesias
4,Angela Padilla,"[Master of Science - MS, Physics Teacher Educa...",[The City College of New York],[],[],https://www.linkedin.com/in/angela-padilla-985...
5,"Dorean Flores, MPA, CIP",[],[],[],[],https://www.linkedin.com/in/doreanflores
6,Anthony Cruz-Balberdy,"[Doctor of Philosophy (Ph.D.), Chemistry - Mol...","[The Graduate Center, City University of New Y...","[Ventus Therapeutics · Full-time, OpenEye Scie...","[Senior Scientist, Summer Intern, Summer Inter...",https://www.linkedin.com/in/anthony-cruz-balbe...
7,Andrew Wills,"[Bachelor’s Degree, Applied Mathematics, Assoc...","[CUNY New York City College of Technology, CUN...","[Brooklyn Public Library, Safe In My Brothers ...","[Consultant, Tutor, Treasurer, Research Assist...",https://www.linkedin.com/in/andrew-wills-67a1a289


In [21]:
CUNY_m1['edu hist']=CUNY_m1.apply(lambda x: combine_two_columns(x.deg, x.school), axis=1)
CUNY_m1['exp hist']=CUNY_m1.apply(lambda x: combine_two_columns(x.titles, x.work), axis=1)
CUNY_m1

 @ 
Juris Doctor, Law @ New England Law | Boston
Masters, Chemical Engineering @ The City College of New York
Bachelors, Chemical Engineering @ The City College of New York
 @ 
Master of Arts (M.A.), Economics @ The New School
Bachelor of Business Administration - BBA, Economics @ Baruch College
Associate of Arts (A.A.), Liberal Arts and Sciences/Liberal Studies @ Borough of Manhattan Community College
Master of Science - MS, Physics Teacher Education @ The City College of New York
 @ 
Doctor of Philosophy (Ph.D.), Chemistry - Molecular Biophysics @ The Graduate Center, City University of New York
Master of Science - MS, Chemistry @ University of Puerto Rico-Mayaguez
Bachelor of Science - BS, Chemistry @ University of Puerto Rico-Mayaguez
Bachelor’s Degree, Applied Mathematics @ CUNY New York City College of Technology
Associate’s Degree, Computer Science @ CUNY New York City College of Technology
Associate of Arts and Sciences (A.A.S.) @ CUNY New York City College of Technology
Comput

/var/folders/c3/wqq2zhw90l596jqv25j63ggc0000gn/T/ipykernel_10825/1829415968.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CUNY_m1['edu hist']=CUNY_m1.apply(lambda x: combine_two_columns(x.deg, x.school), axis=1)
/var/folders/c3/wqq2zhw90l596jqv25j63ggc0000gn/T/ipykernel_10825/1829415968.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CUNY_m1['exp hist']=CUNY_m1.apply(lambda x: combine_two_columns(x.titles, x.work), axis=1)


,Name,deg,school,work,titles,URL,edu hist,exp hist
0,Lori Collins,[],[],"[US Army Futures Command, CCDC · Full-time]",[Computer Engineer],https://www.linkedin.com/in/lori-collins-a6411...,[ @ ],"[Computer Engineer @ US Army Futures Command, ..."
1,Vaughn-Clai Marsh,"[Juris Doctor, Law, Masters, Chemical Engineer...","[New England Law | Boston, The City College of...","[Bryan Cave Leighton Paisner LLP · Full-time, ...","[Compliance and Conflicts Attorney, Senior Con...",https://www.linkedin.com/in/vaughn-clai-marsh-...,"[Juris Doctor, Law @ New England Law | Boston,...",[Compliance and Conflicts Attorney @ Bryan Cav...
2,Allie O.,[],[],[],[],https://www.linkedin.com/in/allieorellana,[ @ ],[ @ ]
3,Ana Alas Iglesias,"[Master of Arts (M.A.), Economics, Bachelor of...","[The New School, Baruch College, Borough of Ma...",[Baruch College · Full-time],[Sponsored Programs Coordinator],https://www.linkedin.com/in/aalasiglesias,"[Master of Arts (M.A.), Economics @ The New Sc...",[Sponsored Programs Coordinator @ Baruch Colle...
4,Angela Padilla,"[Master of Science - MS, Physics Teacher Educa...",[The City College of New York],[],[],https://www.linkedin.com/in/angela-padilla-985...,"[Master of Science - MS, Physics Teacher Educa...",[ @ ]
5,"Dorean Flores, MPA, CIP",[],[],[],[],https://www.linkedin.com/in/doreanflores,[ @ ],[ @ ]
6,Anthony Cruz-Balberdy,"[Doctor of Philosophy (Ph.D.), Chemistry - Mol...","[The Graduate Center, City University of New Y...","[Ventus Therapeutics · Full-time, OpenEye Scie...","[Senior Scientist, Summer Intern, Summer Inter...",https://www.linkedin.com/in/anthony-cruz-balbe...,"[Doctor of Philosophy (Ph.D.), Chemistry - Mol...",[Senior Scientist @ Ventus Therapeutics · Full...
7,Andrew Wills,"[Bachelor’s Degree, Applied Mathematics, Assoc...","[CUNY New York City College of Technology, CUN...","[Brooklyn Public Library, Safe In My Brothers ...","[Consultant, Tutor, Treasurer, Research Assist...",https://www.linkedin.com/in/andrew-wills-67a1a289,"[Bachelor’s Degree, Applied Mathematics @ CUNY...","[Consultant @ Brooklyn Public Library, Tutor @..."


In [23]:
CUNY_m1.to_csv('CUNY_ls_MissingPPL.csv')